In [9]:
import pandas as pd
import spatialpandas as spd
import geopandas as gpd
import rioxarray
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import math
import multiprocessing as mp
import time
import logging
logging.basicConfig(level=logging.INFO)
import landscan_census as lc

# Landscan to Census Tract Join

Load the landscan 3 arc second rasters

In [2]:
tx_night, tx_day = lc.load_landscan(
    night_path='/Users/kpierce/landscan/conus_night_split/conus_night25.TIF',
    day_path='/Users/kpierce/landscan/conus_day_split/conus_day25.TIF'
)

Load the census tract shapefile

In [3]:
tracts = gpd.read_file('/Users/kpierce/CooksProTX/spatial/tigris/texas_census_tracts/census_tracts_2019.shp')

Instantiate a custom class (from `landscan_census.py`) for manipulating the landscan and tract data

In [4]:
ll = lc.Landscan(night_raster=tx_night, day_raster=tx_day, shp=tracts)

Calculate the average population

In [5]:
ll.average_population()

Perform a spatial sort to speed up the eventual spatial join. The `Landscan.dask_spatial_sort` method saves the output data to the parquet format and then reloads them for lazy computation with dask in downstream steps.

In [6]:
ll.dask_spatial_sort(savepath='/Users/kpierce/landscan/workflow_testing/texas_3arcsecond_spatial_sort.parquet')

INFO:root:Spatial sort required 873.3697950839996 seconds.


In [7]:
ll.average_raster_dask

,position,band,x,y,spatial_ref,night_population,day_population,average_population
npartitions=208,,,,,,,,
,point[float64],int64,float64,float64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


Convert the census tract `geopandas.GeoDataFrame` to a `spatialpandas.GeoDataFrame`

In [10]:
ll.gpd_to_spd()

In [11]:
ll.spdf.head()

,GEOID,geometry
0,48439121609,"Polygon([[-97.233694, 32.671749999999996, -97...."
1,48187210708,"Polygon([[-98.26254, 29.578723999999998, -98.2..."
2,48201550301,"Polygon([[-95.43726699999999, 29.981033, -95.4..."
3,48201550302,"Polygon([[-95.463687, 30.007089999999998, -95...."
4,48157675800,"Polygon([[-96.088854, 29.60166, -96.088912, 29..."


Perform the spatial join on the sorted raster data. This join uses `dask`; the `.compute()` method forces the join calculation on the lazily-loaded spatially sorted data.

In [12]:
rdf = spd.sjoin(ll.average_raster_dask, ll.spdf, how='inner').compute()

In [14]:
rdf = rdf.reset_index()

In [15]:
rdf.head()

,hilbert_distance,position,band,x,y,spatial_ref,night_population,day_population,average_population,index_right,GEOID
0,110794072,"Point([-100.077083433025, 28.12874998448501])",1,-100.077083,28.128750,0,0,0,0,4374,48479001710
1,110794080,"Point([-100.077916766355, 28.12958331781501])",1,-100.077917,28.129583,0,0,0,0,4374,48479001710
2,110794084,"Point([-100.077083433025, 28.12958331781501])",1,-100.077083,28.129583,0,0,0,0,4374,48479001710
3,110794091,"Point([-100.077083433025, 28.13041665114501])",1,-100.077083,28.130417,0,0,0,0,4374,48479001710
4,110794095,"Point([-100.077916766355, 28.13041665114501])",1,-100.077917,28.130417,0,0,0,0,4374,48479001710


In [30]:
rdf = rdf.set_index('hilbert_distance')

In [24]:
rdf.shape

(95118149, 10)

Over 95 million rows, yikes.

In [26]:
type(rdf)

spatialpandas.geodataframe.GeoDataFrame

Save this computationally expensive dataset

In [10]:
from spatialpandas.io import to_parquet, read_parquet_dask, read_parquet

In [31]:
rdf.to_parquet('/Users/kpierce/landscan/workflow_testing/texas_3arcsecond_tract_join_flat.parquet')

# Landscan to SVI join on Census Tract FIPS

## Interpolation for persons, housing units, and households

Landscan provides us with a more accurate and more granular understanding of population distribution. The ACS provides us with coarse estimates of population distribution, and counts and percentages of the population falling into certain demographic categories.

We calculate the count and percent of the population in different demographic categories at 30 arcsecond resolution under the following formula:

Let $j$ = { *landscan cells* }, $k$ = { *census tracts* }, and $m$ = { *demographic subset variables from the ACS* }. 

The total population of a census tract estimated from landscan data is N$_{k}$ = $\sum$N$_{jk}$ for $j$ landscan cells ${j}$ associated with census tract $k$ (1). 

The population weight (the fraction of the tract population in each landscan cell) is calculated as weight$_{jk}$ = N$_{jk}$ / N$_{k}$ (2). 

Following equations 1 and 2, the total number in each demographic category for each landscan cell is calculated as N$_{jm}$ = N$_{km}$ * weight$_{jk}$ for demographic category $m$ and census tract $k$ (3). 

Finally, the percentage of the population in each demographic category in each landscan cell is calculated as percent$_{jm}$ = min(100, N$_{jm}$ / N$_{jk}$) (4).

Equation 4 accounts for the possibility that a census-derived population estimate in a demographic category may be larger than a landscan derived population total. These anomalies need further investigation, so the uncorrected percentages, which may range greater than 100%, are included in the outputs.

In some cases, only a single landscan cell centroid will fall within a census tract. We do not consider that some fraction of the landscan cell may be outside of these census tracts in conducting the following interpolations.

## Dollars

We do not perform interopolation on variables measured in dollars.

In [11]:
import sqlite3

In [12]:
db_name = '/Users/kpierce/protxdb/data/db/cooks_20210923.db'
db_conn = sqlite3.connect(db_name)
db_cursor = db_conn.cursor()

Solve equation 1.

In [13]:
rdf = read_parquet('/Users/kpierce/landscan/workflow_testing/texas_3arcsecond_tract_join_flat.parquet')

In [14]:
tract_totals = rdf[['GEOID', 'average_population']].groupby('GEOID').agg({'average_population': sum}).reset_index()

In [15]:
tract_totals.shape

(5265, 2)

In [16]:
rdf_tract = pd.merge(rdf, tract_totals, on='GEOID', how='left')

In [39]:
rdf_tract.head()

,position,band,x,y,spatial_ref,night_population,day_population,average_population_x,index_right,GEOID,average_population_y
0,"Point([-100.077083433025, 28.12874998448501])",1,-100.077083,28.128750,0,0,0,0,4374,48479001710,2982
1,"Point([-100.077916766355, 28.12958331781501])",1,-100.077917,28.129583,0,0,0,0,4374,48479001710,2982
2,"Point([-100.077083433025, 28.12958331781501])",1,-100.077083,28.129583,0,0,0,0,4374,48479001710,2982
3,"Point([-100.077083433025, 28.13041665114501])",1,-100.077083,28.130417,0,0,0,0,4374,48479001710,2982
4,"Point([-100.077916766355, 28.13041665114501])",1,-100.077917,28.130417,0,0,0,0,4374,48479001710,2982


Solve equation 2.

In [17]:
rdf_tract['population_weight'] = rdf_tract['average_population_x'] / rdf_tract['average_population_y']

Solve equation 3.

In [18]:
people = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.UNITS = 'count' and \
        d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE in ('persons commuting', 'persons') and \
        d.DEMOGRAPHICS_NAME <> 'TOTPOP'
    """, db_conn)

hh = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.UNITS = 'count' and \
        d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE = 'households' and \
        d.DEMOGRAPHICS_NAME <> 'HH'
    """, db_conn)

hu = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.UNITS = 'count' and \
        d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE = 'housing units' and \
        d.DEMOGRAPHICS_NAME <> 'HU'
    """, db_conn)

dollars = pd.read_sql_query(
    """select * from demographics d \
    join display_data dd on d.DEMOGRAPHICS_NAME = dd.NAME 
    where d.YEAR = 2019 and \
        d.GEOTYPE = 'tract' and \
        dd.UNIT_OF_MEASURE = 'dollars'
    """, db_conn)

In [19]:
demo_data = pd.concat(
    [
        people[['GEOID', 'DEMOGRAPHICS_NAME', 'VALUE']], 
        hh[['GEOID', 'DEMOGRAPHICS_NAME', 'VALUE']],
        hu[['GEOID', 'DEMOGRAPHICS_NAME', 'VALUE']]
    ]
)
data_wide = pd.pivot(demo_data, index='GEOID', columns='DEMOGRAPHICS_NAME', values='VALUE').reset_index()

In [20]:
data_wide.head()

DEMOGRAPHICS_NAME,GEOID,10_14_MIN,15_19_MIN,20_24_MIN,25_29_MIN,30_34_MIN,35_39_MIN,40_44_MIN,45_59_MIN,5LESS_MIN,...,NOVEH,OTHER_RACE_ALONE,POV,RENTER_OCCUPIED_HU,TOTAL_COMMUTE_POP,TWO_OR_MORE_RACES,UNEMP,UNINSUR,WHITE_ALONE,WHITE_ALONE_NOT_HISPANIC_LATINO
0,48001950100,138.0,147.0,222.0,105.0,368.0,127.0,59.0,162.0,64.0,...,52.0,59.0,780.0,210.0,1672.0,114.0,53.0,533.0,4341.0,3823.0
1,48001950401,0.0,16.0,18.0,11.0,0.0,5.0,0.0,10.0,39.0,...,0.0,18.0,0.0,58.0,145.0,43.0,0.0,79.0,2872.0,1678.0
2,48001950402,23.0,4.0,13.0,0.0,0.0,0.0,0.0,0.0,111.0,...,0.0,0.0,44.0,97.0,163.0,15.0,4.0,11.0,4400.0,2664.0
3,48001950500,548.0,268.0,115.0,234.0,102.0,56.0,0.0,118.0,56.0,...,138.0,281.0,854.0,667.0,1900.0,164.0,113.0,658.0,2982.0,1613.0
4,48001950600,642.0,315.0,271.0,45.0,222.0,11.0,36.0,106.0,134.0,...,293.0,0.0,880.0,648.0,2251.0,54.0,109.0,1035.0,3549.0,3384.0


In [21]:
calc_columns = data_wide.columns[data_wide.columns != 'GEOID']

In [22]:
calc_columns

Index(['10_14_MIN', '15_19_MIN', '20_24_MIN', '25_29_MIN', '30_34_MIN',
       '35_39_MIN', '40_44_MIN', '45_59_MIN', '5LESS_MIN', '5_9_MIN',
       '60_89_MIN', '90PLUS_MIN', 'AGE17', 'AGE65',
       'AMERICAN_INDIAN_ALASKA_NATIVE_ALONE', 'ASIAN_ALONE',
       'BLACK_AFRICAN_AMERICAN_ALONE', 'CROWD', 'DISABL', 'FOREIGN_BORN',
       'GROUPQ', 'HISPANIC_LATINO', 'LIMENG', 'MINRTY', 'MOBILE',
       'NATIVE_HAWAIIAN_OTHER_PACIFIC_ISLANDER_ALONE', 'NOHSDP', 'NOVEH',
       'OTHER_RACE_ALONE', 'POV', 'RENTER_OCCUPIED_HU', 'TOTAL_COMMUTE_POP',
       'TWO_OR_MORE_RACES', 'UNEMP', 'UNINSUR', 'WHITE_ALONE',
       'WHITE_ALONE_NOT_HISPANIC_LATINO'],
      dtype='object', name='DEMOGRAPHICS_NAME')

From here the joins are too expensive and require Frontera.

In [23]:
rdf_tract_svi = pd.merge(rdf_tract, data_wide, on='GEOID', how='left')

KeyboardInterrupt: 